# Auditing a Classifier for Fairness Based on Movement Patterns

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle

from scipy.special import xlogy, xlog1py
from joblib import Parallel, delayed
from tqdm import tqdm

from pathlib import Path

### Bernoulli-based spatial scan statistic parameters

In [ ]:
num_simulations = 500 # Number of Monte Carlo simulations to derive an approx. distribution of the test statistics
alpha = 0.05          # Significance level required.

### Main code

Read a dictionary containing the candidates (subsets of cells with at least an associated object) of all the grids; the candidates are in a 1D flattened array.

In [ ]:
path_candidates = './data_simulator/huge_dataset/gencand/'

with open(path_candidates + "dict_flattened_candidates.pkl", "rb") as f:
    data = pickle.load(f)

### Load the dictory containing the flattened objects ID lists associated with the candidates ###
flat_ids, indptr, lenghts = data['flat_ids'], data['start_pos'], data['lengths']
del data

# Ensure the big arrays are contiguous (helps memmap efficiency)
flat_ids = np.ascontiguousarray(flat_ids)
indptr   = np.ascontiguousarray(indptr)
lenghts  = np.ascontiguousarray(lenghts)

# Check that there is no candidate with 0 associated objects. It shouldn't happen, but we do a quick sanity check here.
assert np.all(lenghts == np.diff(indptr)), "lenghts/indptr mismatch (or empty segments present)"
assert np.all(lenghts > 0), "Candidates with zero associated objects detected, should not happen!"

### Read the dataset with the objects' "true" labels.

**TODO**: we are using dummy labels for now.

In [ ]:
# Read the dataset containing the true labels of the objects.
n_objects = 100000
positive_rate = 0.6
labels = np.random.default_rng(42).binomial(n=1, p=positive_rate, size=n_objects).astype(np.int8)
# labels

#### Compute the empirical distribution of the considered test statistic with a certain number of Monte Carlo simulations.

Here we perform the Monte Carlo simulations needed to determine the distribution of the test statistics under the assumption that the null hypothesis is true. The test statistics used is the maximum log likelihood ratio computed across the regions of all the grids, while the likelihood function is the Bernoulli-based one.

In [ ]:
import numba as nb

# Segmented reduction function used within 'batch_max_likelihood_ratio'
@nb.njit(cache=True, nogil=True, fastmath=True)
def segmented_sum_indirect(labels_objects, flat_ids, indptr):
    n = indptr.size - 1
    out = np.empty(n, dtype=np.uint32)
    for i in range(n):
        s = 0
        start = indptr[i]
        end   = indptr[i + 1]
        for p in range(start, end):
            s += labels_objects[flat_ids[p]]
        out[i] = s
    return out

def batch_max_likelihood_ratio(labels_objects: np.ndarray, 
                               flat_ids: np.ndarray, indptr: np.ndarray, lenghts: np.ndarray,
                               tot_sum_labels: int,
                               logL0_max: float) -> tuple[np.ndarray, np.ndarray, np.ndarray, float]:    

    # Gather labels for all ids, then sum per candidate via segmented reduction.
    #
    # NOTE: this version instantiates an output array of the same size of 'flat_ids', so it can be large!
    # flat_vals = labels_objects[flat_ids]
    # inside_sum = np.add.reduceat(flat_vals, indptr[:-1]).astype(np.uint32, copy=False)
    #
    # NOTE: this version computes the segmented reduction on the fly. It is faster and avoids allocating
    #       large 'flat_vals' arrays.
    inside_sum = segmented_sum_indirect(labels_objects, flat_ids, indptr)

    # Vectorized computation: for each candidate subset of cells, compute the positive rate of the objects
    # associated with the subset vs the positive rate of the other objects.
    # NOTE: we use np.divide with the `where` parameter to avoid divisions by zero.
    p, n = inside_sum, lenghts
    P, N = tot_sum_labels, labels_objects.size
    inside_positive_rate  = np.divide(p, n, out=np.zeros_like(p, dtype=np.float32), where=(n > 0))
    outside_positive_rate = np.divide(P - p, N - n, out=np.zeros_like(p, dtype=np.float32), where=((N - n) > 0))
    

    # Potentially numpy-unsafe computation of the log-likelihood under the alternative hypotesis.
    #logL1 = (p * np.log(inside_positive_rate)
    #         + (n - p) * np.log1p(-inside_positive_rate)
    #         + (P - p) * np.log(outside_positive_rate)
    #         + (N - n - (P - p)) * np.log1p(-outside_positive_rate))
    
    
    # Safe alternative computation of the log-L1 via scipy functions. 
    # NOTE: the log-likelihood is -inf when the positive rate is 0 or 1, which can happen when p==0 or p==n for the inside positive rate, 
    # or when P-p==0 or N-n-(P-p)==0 for the outside positive rate. This is not a problem per se, since we are interested in the likelihood
    # ratio, and if the likelihood under the alternative hypotesis is -inf, then the likelihood ratio will be 0, which is what we expect in
    # these cases.
    # valid = (n > 0) & (n < N) # optional: mask degenerate windows (n==0 or n==N)
    # logL1 = np.full_like(inside_positive_rate, -np.inf, dtype=np.float32)
    logL1 = ( xlogy(p, inside_positive_rate) + 
              xlog1py((n - p), -inside_positive_rate) +
              xlogy((P - p), outside_positive_rate) +
              xlog1py((N - n - (P - p)), -outside_positive_rate) )

    # Vectorized computation of the log-likelihood ratio of the candidates
    # logLR = logL1 - logL0_max
    logL1 -= logL0_max
    maxLogLR = float(np.nanmax(logL1))

    return inside_positive_rate, outside_positive_rate, logL1, maxLogLR

In [ ]:
import math
import numpy as np
import numba as nb

@nb.njit(cache=True, nogil=True)
def max_loglr_streaming(is_simulation,
                        labels, 
                        flat_ids, indptr, lengths, 
                        P, logL0_max):
    N = labels.size
    max_lr = -np.inf
    m = lengths.size
    lr_candidates = np.empty(m if not is_simulation else 0)
    in_rate_candidates = np.empty(m if not is_simulation else 0)
    out_rate_candidates = np.empty(m if not is_simulation else 0)
    for i in range(m):
        start = indptr[i]
        end   = indptr[i + 1]

        # segmented sum without allocating flat_vals
        p = 0
        for k in range(start, end):
            p += labels[flat_ids[k]]

        n = lengths[i]
        if (n <= 0) or (n >= N) : continue  # degenerate candidate (no object or contains all objects)

        # Positive rate for the objects associated with the candidate, and for the objects that are not.
        inside_rate  = p / n
        outside_rate = (P - p) / (N - n)

        # compute logL1 safely (xlogy/xlog1py equivalents)
        logL1 = p * math.log(inside_rate) if p > 0 else 0.0
        logL1 += (n - p) * math.log1p(-inside_rate) if n - p > 0 else 0.0
        #
        Pout = P - p
        Nout = N - n
        logL1 += Pout * math.log(outside_rate) if Pout > 0 else 0.0
        #
        neg_out = Nout - Pout
        logL1 += neg_out * math.log1p(-outside_rate) if neg_out > 0 else 0.0

        # Compute the log-likelihood ratio.
        lr = logL1 - logL0_max
        
        # Add the ratio to the vector of log-LRs of the candidates.
        if (not is_simulation) : 
            lr_candidates[i] = lr
            in_rate_candidates[i], out_rate_candidates[i] = inside_rate, outside_rate

        # Update the candidate with the largest log-LR found so far.
        if lr > max_lr:
            max_lr = lr

    return in_rate_candidates, out_rate_candidates, lr_candidates, max_lr


@nb.njit(cache=True, nogil=True, parallel=True)
def compute_simulations(num_sims, labels, 
                        flat_ids, indptr, lengths,
                        P, logL0_max):
    
    vec_max_LR = np.empty(num_sims, dtype=np.float32)
    for s in nb.prange(num_sims) :
        # Shuffle the labels.
        np.random.seed(s)
        shuffled_labels = labels.copy()
        np.random.shuffle(shuffled_labels)

        # Compute the max log-LR distribution expected under the assumption that H_0 is true.
        vec_max_LR[s] = max_loglr_streaming(True, shuffled_labels, flat_ids, indptr, lengths, P, logL0_max)[-1]

    return vec_max_LR

In [ ]:
### PARALLEL NUMBA VERSION ###

# Compute the L_0 likelihood, which models the likelihood of observing the labels in the data under the the assumption that the 
# null hypotesis H_0 is true, i.e., there is a single global distribution that governs the labels. L_0 is constant across 
# permutations, since it depends only on the total number of positive and negative labels in the dataset, which does not change
# when shuffling the original labels.
P = labels.sum(dtype=np.uint32) # Constant across permutations
N = labels.size
rho = P / N
logL0_max = P * np.log(rho) + (N - P) * np.log1p(-rho)

# Compute the simulations' max log-LRs in parallel via numba.
vec_max_LR = compute_simulations(num_simulations, labels, 
                                 flat_ids, indptr, lenghts, 
                                 P, logL0_max)

In [ ]:

# Sort the max_LR distribution obtained with the simulations.
# This is the distribution of the likelihood ratios of the most extreme regions observed empirically
# under the assumption that the null hypothesis H_0 is true.
sorted_vec_max_LR = np.sort(vec_max_LR)

# Print the most extreme likelihood ratio observed across all the simulations.
print(sorted_vec_max_LR[-1])

#### Compute the max log likelihood ratio from the candidates when considering the original labels.

In [ ]:
#inside_positive_rate, outside_positive_rate, vec_LR_dataset, max_LR_dataset = \
#    batch_max_likelihood_ratio(labels,
#                               flat_ids, indptr, lenghts,
#                               P, logL0_max)

inside_positive_rate, outside_positive_rate, vec_LR_dataset, max_LR_dataset = \
    max_loglr_streaming(False, labels, flat_ids, indptr, lenghts, P, logL0_max)

# DEBUG
inside_positive_rate, outside_positive_rate, vec_LR_dataset, max_LR_dataset

#### Determine if $H_0$ must be rejected.

In [ ]:
# Determine where the max LR computed with the original labels fall in the empirical test statistic's distribution.
rank = np.count_nonzero(vec_max_LR >= max_LR_dataset)

# Monte Carlo p-value of the observed test statistic's value derived from the ranked empirical test statistic's distribution 
# (right tail), with +1 correction to include max_LR_dataset itself.
p_value = (rank + 1) / (num_simulations + 1)

# Based on the distribution and the real data we have, decide if we have to reject H_0.
reject_H0 = p_value <= alpha

print(f"Statistical significance alpha: {alpha}")
print(f"Position in sorted MC sample: {rank}/{num_simulations} (extreme if below position {int(num_simulations * alpha)})")
print(f"Monte Carlo p-value: {p_value:.6f}")
print(f"Decision: {'Reject H0' if reject_H0 else 'Do NOT reject H0'}")

### Various DEBUGS and SANITY CHECKS

In [ ]:
# DEBUG and SANITY CHECK: plot the distribution of the maximum likelihood ratios computed in the Monte Carlo simulations.
#
# Lots of simulations produce “modest” max log-LR, and a few simulations produce unusually large max LLRs.
# So, we expect to see a concentration of max LLRs on the left, with a long right tail.

import numpy as np
import matplotlib.pyplot as plt

plt.hist(vec_max_LR, bins=50)
plt.xlabel("Values of the simulations' max likelihood ratios")
plt.ylabel(f"Frequency (log-scale) (total: {vec_max_LR.size} simulations)")
plt.yscale("log")
plt.show()

In [ ]:
# DEBUG: plot the distribution of the likelihood ratios computed over the subsets of cells when considering the original labels.
plt.hist(vec_LR_dataset, bins=200)  # tune bins (100–500 usually fine)
plt.xlabel("Values of the candidates' likelihood ratios")
plt.ylabel(f"Frequency (log-scale) (total: {vec_LR_dataset.size} objects)")
plt.yscale("log")
plt.show()

In [ ]:
# DEBUG: find the characteristics of the subset of cells with the maximum LR when considering the original labels.

# Find where the candidate with the max LR is located.
idx = np.argsort(vec_LR_dataset)

# Sort the 1D arrays of interest accordingly.
vec_LR_dataset_sorted = vec_LR_dataset[idx]
lens_sorted = lenghts[idx]
inside_positive_rate_sorted = inside_positive_rate[idx]
outside_positive_rate_sorted = outside_positive_rate[idx]

# Print the max LR candidate's info.
print(f"Info most 'problematic' candidate: local ps: {inside_positive_rate_sorted[-1]}, " +
      f"other ps: {outside_positive_rate_sorted[-1]}, num_objs_candidate: {lens_sorted[-1]}, " +
      f"likelihood ratio: {vec_LR_dataset_sorted[-1]}")
